### The Titanic

![The Titanic](titanic.jpeg)

#### Context

So what do we know already?:

- Most of you will probably have seen the movie by James Cameron (starring Kate Winslet and Leonardo DiCaprio, 1997) which is one of several movies made with the disaster as subject;
- The disaster took place during the night of April 14th, 1912 when the ship hit an iceberg on her mayden voyage out of Southampton (UK) to New York (USA) via Cherbourg (FR) and Queenstown (IRE);
- The loss of lives was 1501 out of a total of 2207 passengers and crew;
- There was a shortage of lifeboats and some of the lifeboats were set afloat whilst not fully occupied.

#### The data

Now we are going to cheat bigtime here, because we will use the dataset(s) prepared for the Kaggle competition. In the wild, out there, you will have to collect the data yourself, often from various sources, messy and incomplete and you will have to spend considerable time searching for data and cleaning the data you found. Estimations vary, but one will probably spend between 60-80% of one's time on searching and cleaning.

Re-use of data, as we are doing here, can be a big timesaver. Hence the importance of "tidy data".

#### Get to know the data

I usually load the data, if it is data in csv format, in my editor and poke aroud a little bit:

In [ ]:
!/usr/local/bin/aquamacs /Users/peter/Documents/bootcamps/faector_bc/data/titanic3.csv

All sorts of useful insights can be get this way:

- the name column also contains some extras, like titles: Mr., Mrs., Miss., Master, Rev., Don., etc. as well as the maiden names of female passengers. Passenger 152, for example, is Mrs. Thomas Pearce == Edith Wearne;
- whenever we see two adjacent comma's ',,' we know we are dealing with missing data;
- we might wonder what the column headers "sibsp" and "parch" mean: the number of SIBlings / SPouses abord, the number of PARents / CHildren aboard.
- Suppose, we want to predict who had the best chances of survival: men or women, younger persons or older persons, people travelling first class? What could be a sound strategy? Doe we have some ideas?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.style.use('ggplot')
%matplotlib inline

In [ ]:
df_titanic = pd.read_csv("data/titanic3.csv", sep=";")

Now that we have our csv file as a Pandas dataframe, we can use methods on the Pandas "dataframe object" that the library provides:

- [dataframe].shape: Presents the dimensions of the dataframe as a tuple
- [dataframe].head(): Shows the first 5 rows of the dataframe
- [dataframe].describe(): Generates some descriptive statistics
- [dataframe].info(): Generates overview of columns and datatypes and their values

In [ ]:
df_titanic.shape

In [ ]:
df_titanic.head(10)

In [ ]:
df_titanic.describe()

In [ ]:
df_titanic.info()

Often it can save time later on to have a careful look at the output of the info() method. Here, for example, we can see that the 'age' column has a dtype of type object, but we would expect a number, don't we.

Let's change that right away. It happens often that data from csv files is read into a Pandas dataframe as strings where we know we are dealing with numbers of a certain type (integers or floats). So Pandas has a method to do the cleaning up. Let's do that right away:

In [ ]:
pd.to_numeric(df_titanic.age)

We still have "NaN's" or missing values, but we will deal with those later on.

What do we want to know?
It might well be you have some questions in advance and want to use the data to see if you can find answers, or it could well be you want to play a little bit with the data in order to come up with questions.

Both approaches suggest the following steps:

- Explore the data (load it, look at it)
- Clean the data (missing values, splitting columns, etc.)
- Plot (try to visualize correlations, insights, ...)
- Assumptions (try to formulate hypotheses, rinse and repeat)

Let's look at the survivors. We know that 38% of the passengers survived the disaster; 62% died. Can we find out somewhat more about the chance of survival?

For the Kaggle competition, since the competition is about machine learning, you would work with two csv files, a training set and a test set. You would learn the computer to predict the survival of the people in the test set using the data of the training set.

A safe bet to start with would be: 62% of the people in the test set died. But even without machine learning, we can do better. Back to the data.

Suppose we want to have a look at the survival rate in combination with the data about the sex, age, and class of the passengers?

In [ ]:
df_titanic[['survived', 'age', 'sex', 'pclass']][0:15]

Bummer, we are just looking at people that travelled first class. The file must be sorted that way. Let's look at the other end of the passenger list:

In [ ]:
df_titanic[['survived', 'age', 'sex', 'pclass']][-15:]

Right, we definitely have something here: Seems your chances of survival were much lower when travelling third class, but we have to be careful, we have just the data of three females in this last sample. And being male is not a good thing so it seems.

When we dive in deeper, we often have to do something with the data. When looking at the age of passengers, we see that we have missing data ("NaN's"). The info() method already showed this 1046 non-null entries for age, which boils down to the fact that we miss 1309 - 1046 = 263 entries for age. We should do something about this.

We might as well drop some columns, like ticket, cabin, boat, home.dest, and body, because we probably are not going to use them.

In [ ]:
# Dropping the columns for Cabin, Ticket, Body, Boat, and Home.dest
df_titanic = df_titanic.drop(['cabin', 'ticket', 'body', 'boat', 'home.dest'], axis=1)

We can use the Pandas crosstab method to zoom in on the relation between the sex of passengers and the survival rate:

In [ ]:
df_titanic.info()

In [ ]:
pd.crosstab(df_titanic.sex, df_titanic.survived)

In [ ]:
print(df_titanic.groupby('sex').survived.median())

If we try to get a quick insight, using the same method, on the relation between survival and age, things are different:

In [ ]:
pd.crosstab(df_titanic.age, df_titanic.survived)

One way to getter a better insight into the relation between survival and age would be to group the passengers by age-group. We could do this by using a small program to change the age column (gaining some information, but losing the precise ages; we could use the age column to add an age-group column in order to side-step the dilemma). But there is a better way, we can use a visualization in which we divide the entries in the age column over so-called bins. For this we use the library matplotlib.

But before we can visualize things, we need to repair the missing values in the 'age' column. You know more about this than we do, but we will use the median of the ages we have values for.

In [ ]:
impute_value = df_titanic['age'].median()
impute_value

Then we replace the missing values with the median (28.0).

In [ ]:
df_titanic['age'] = df_titanic['age'].fillna(impute_value)

Our dataframe should now contain 1309 values in the 'age' column. Let's check it:

In [ ]:
df_titanic.info()

We can basically do two things here if we are interested in the relation between 'age' and survival rate:

1) We can use a visualization and drop all the values in the 'age' column into a number of bins -- that way we can get an insight into the relation;
2) Or we can use the values in the 'age' column to add a new column 'age_group' to the dataframe assigning passengers to age groups.

In [ ]:
figure = plt.figure(figsize=(13,8))
plt.hist([df_titanic[df_titanic['survived']==1]['age'],df_titanic[df_titanic['survived']==0]['age']],
         stacked=True, color = ['g','r'],
         bins = 20,label = ['Survived','Dead'])
plt.xlabel('Age')
plt.ylabel('Number of passengers')
plt.legend()

The crosstab and this chart seem to confirm that one old code of conduct that sailors and captains follow in case of threatening situations: "Women and children first!".

In [ ]:
# Let's add the 'age_group' column to our dataframe
age_labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
df_titanic['age_group'] = pd.cut(df_titanic.age, range(0, 81, 10), right=False, labels=age_labels)

In [ ]:
# Let's inspect the new dataframe
df_titanic.head()

In [ ]:
# Now that we have caught the large number of age values in age groups (categories), we can try crosstab again
pd.crosstab(df_titanic.age_group, df_titanic.survived)

In [ ]:
print(df_titanic.groupby('age_group').survived.mean())

In [ ]:
# And did it matter which class you were travelling?
pd.crosstab(df_titanic.pclass, df_titanic.survived)

In [ ]:
# The same but now proportional
print(df_titanic.groupby('pclass').survived.mean())

For the Kaggle competition the Titanic dataset is randomly split into a training set and a testing set (similar set minus the column 'Survived'). All participants worked with these two sets to predict the survival rate of the test set using patterns in the training set.

Based upon what we have found so far we could predict that all women and all children survive, and our predictions > 75% correct. There are many notebooks published with strategies tackling the now classic Kaggle Titanic competiton and, because most of them use ML methods, I have studied several of them. They easily predict > 80% correctly.

The main strategies used are defining categories out of the "raw" data that will enable the ML algorithms to better predict the survival rate of the test set. For example: Construct age groups -- just as we did with the visualization above -- and often changing the training dataset. Or by looking at people travelling together in various ways, etc. By looking at pclass in combination with fare paid and sometimes in combiantion with the titles that are available in the name field ("Women and children first, and it helped when you were travelling first class"). 

Almost all notebooks show that competitors rely heavily on visualizations in order to fathom whether they are on to something when working with patterns in the data.

In the following we will walk trough a very simple example of ML with scikit-learn.

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [ ]:
train[:4]

Usually libraries like statsmodels and scikit-learn can not deal with missing values, so we have a look at the columns to see whether they contain missing data.

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

We need Age as a predictor, so we have to fill in the missing values. There are a number ways to do it, but we just use the median of the Age column in the training set:

In [ ]:
impute_value = train['Age'].median()
impute_value

In [ ]:
# We use that value to fill in all the missing values, both in the training as well as in the test set.
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = train['Age'].fillna(impute_value)

In [ ]:
train.isnull().sum()

Often ML models do not work/learn well with strings as values. In the Sex column we now have "Male" and "Female". We are going to use that information to fill a new column "IsFemale" with 1 if the person has a Sex column with a value of "female".

In [ ]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [ ]:
train['IsFemale'].head()

In [ ]:
# We choose our model variables: 'IsFemale', 'Age', and 'Pclass'
predictors = ['IsFemale', 'Age', 'Pclass']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values

In [ ]:
X_train[:5]

In [ ]:
y_train[:5]

The whole idea of data science with ML is to choose an appropriate model and using well engineered features. We did not really dive in, the following is just meant to show the principles involved.

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
y_predict[:10]

Of course, we would need to have the real values for 'Survived' in our test set to be able to compare our prediction against what really is/was the case.

    array([0,1,0,0,1,1,0,1,0,0])

Because our 1 or 0 for survival predictions is about the fate of real people, passengers of the Titanic. Let's have a look again at one of the first encounters with the people we had:

In [ ]:
df_titanic.head(10)

[Here is the story](https://www.encyclopedia-titanica.org/titanic-victim/hudson-joshua-creighton.html)

And here is a another curious one. Remember that in one of our visualizations the eldest passenger saved was 80 years of age. Who was that person and what happened?

When we look that person up in our file titanic3.csv, we find that he was a man, who travelled first class, Mr. Algernon Henry Wilson Barkworth. When we look him up on the Titanic website we find that indeed he was a survivor, born in June 1864. Which makes him 47 when the disaster took place. He was 80 when he died on 7 january 1945. So, we can get rid of an outlier in our dataset! Barkworth was 47 when he survived.

[You can read the story here](https://www.encyclopedia-titanica.org/titanic-survivor/algernon-barkworth.html)